In [1]:
from typing import TYPE_CHECKING

import lovely_tensors as lt
import torch
from torch import Tensor, nn

In [2]:
# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all" # type: ignore  # noqa: PGH003

In [3]:
dtype = torch.float32

In [4]:
class Layer(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.weights = nn.Parameter(torch.tensor([[-1, 1], [-2, 2]], dtype=dtype))

    def forward(self, x:Tensor) -> Tensor:
        return torch.matmul(x, self.weights)

    if TYPE_CHECKING:
        def __call__(self, x:Tensor) -> Tensor: ...

In [11]:
data = torch.tensor([1, 2], dtype=dtype)
target = torch.tensor([0, 1], dtype=dtype)

model = Layer()

scripted_model = torch.jit.script(model)
scripted_model.graph

traced_model = torch.jit.trace(model, data)
traced_model.graph

graph(%self : __torch__.Layer,
      %x.1 : Tensor):
  %weights : Tensor = prim::GetAttr[name="weights"](%self)
  %4 : Tensor = aten::matmul(%x.1, %weights) # /tmp/ipykernel_3836931/1800329445.py:7:15
  return (%4)

graph(%self : __torch__.___torch_mangle_4.Layer,
      %x : Float(2, strides=[1], requires_grad=0, device=cpu)):
  %weights : Tensor = prim::GetAttr[name="weights"](%self)
  %5 : Float(2, strides=[1], requires_grad=1, device=cpu) = aten::matmul(%x, %weights) # /tmp/ipykernel_3836931/1800329445.py:7:0
  return (%5)

In [13]:
# SGD

learning_rate = 0.1

#w.grad.data.zero_()

output = model(data)
loss = (output - target).sum()
print(f"loss: {loss}")

loss.backward()

print(model.weights.data)
print(model.weights.grad)

torch.jit.trace(model)

#w -= learning_rate * w.grad.data

loss: -1.0
tensor([[-1.,  1.],
        [-2.,  2.]])
tensor([[2., 2.],
        [4., 4.]])


RuntimeError: example_kwarg_inputs should be a dict